In [2]:
import csv

DATADIRECTORY="./simData"
IMAGEDIRECTORY="./simData/IMG"
DRIVINGLOG=DATADIRECTORY+"/driving_log.csv"
lines=[]
with open(DRIVINGLOG) as csvFile:
    reader=csv.reader(csvFile)
    for line in reader:
        lines.append(line)
print("lines:", len(lines))

import cv2

CENTERIMAGE=0
STEERINGMEASUEMENT=3
CAMERAS=3
# CENTER, LEFT, RIGHT
STERRINGADJUSTMENT=[0, .25, -.25]


lines: 10837


In [3]:
import cv2
import numpy as np
import sklearn

def readImage(imagePathOnRemote):
    filename=imagePathOnRemote.split("/")[-1]
    imagePath=IMAGEDIRECTORY+"/"+filename
    image=cv2.imread(imagePath)
    #print("imagePath:", imagePath, "image.shape:", image.shape)
    return image;

def generator(lines, batchSize=128):
    #print("lines:", len(lines))
    numberOfSamples = len(lines)
    while 1: # Loop forever so the generator never terminates
        lines=sklearn.utils.shuffle(lines) # every epoch
        #print("shuffled lines:", len(lines))
        for offset in range(0, numberOfSamples, batchSize):
            batchOfLines = lines[offset:offset+batchSize]

            images=[]
            steeringMeasurements=[]

            for line in batchOfLines:
                for camera in range(CAMERAS):
                    imagePathOnRemote=line[camera]
                    image=readImage(imagePathOnRemote)
                    if (image is None):
                        print ("line:", line)
                        print("image:", image, ", imagePath:", imagePath)
                        exit
                    images.append(image)
                    steering=float(line[STEERINGMEASUEMENT])+STERRINGADJUSTMENT[camera]
                    steeringMeasurements.append(steering)
                    images.append(cv2.flip(image,1))
                    steeringMeasurements.append(steering*-1)
                    assert len(images)==len(steeringMeasurements)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(steeringMeasurements)
            assert len(X_train)==len(y_train)
            #print("X_train.shape:", X_train.shape, "y_train.shape:", y_train.shape)
            #print("X_train[0].shape:", X_train[0].shape)
            yield sklearn.utils.shuffle(X_train, y_train)

In [4]:
from sklearn.model_selection import train_test_split

train_samples, validation_samples = train_test_split(lines, test_size=0.2)
print("train_samples:", len(train_samples), "validation_samples:", len(validation_samples))
print("train_samples[0]:",len(train_samples[0]))

# compile and train the model using the generator function
train_generator = generator(train_samples, batchSize=128)
validation_generator = generator(validation_samples, batchSize=128)

train_samples: 8669 validation_samples: 2168
train_samples[0]: 7


In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Lambda
from keras.layers import Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
import keras as keras

sampleImage=readImage(lines[0][0])
print("sampleImage.shape:", sampleImage.shape)
XtrainInputShape=sampleImage.shape[0:len(sampleImage.shape)]
print ("XtrainInputShape:", XtrainInputShape)

model = Sequential()
model.add(Lambda(lambda x: (x / 127.5) - 1, input_shape=XtrainInputShape, output_shape=XtrainInputShape))
#model.add(Lambda(lambda x: (x / 127.5) - 1, input_shape=(3,80,320), output_shape=(3,80,320)))

model.add(Cropping2D(cropping=((70,25),(0,0))))

# nvidia model
# keras.layers.normalization.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Convolution2D(24, 5, 5, subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(36, 5, 5, subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(48, 5, 5, subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))

##
model.add(Dense(1))

adamOptimizer=keras.optimizers.Adam(lr=0.0001)
model.compile(loss='mse', optimizer=adamOptimizer)
#model.compile(loss='mse', optimizer='adam')

history_object=model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator,
                                   nb_val_samples=len(validation_samples), nb_epoch=20)

#adamOptimizer=keras.optimizers.Adam(lr=0.0001)
#model.compile(optimizer=adamOptimizer, loss='mse', metrics=['accuracy'])
#history_object=model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=20)

import datetime

modelFilename=datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")+".model"
print ("saving model as:", modelFilename)
model.save(modelFilename)

Using TensorFlow backend.


sampleImage.shape: (160, 320, 3)
XtrainInputShape: (160, 320, 3)
Epoch 1/20
8448/8669 [============================>.] - ETA: 0s - loss: 1.1823

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


9216/8669 [===============================] - 28s - loss: 1.1524 - val_loss: 0.0680
Epoch 2/20
9216/8669 [===============================] - 23s - loss: 0.7123 - val_loss: 0.0651
Epoch 3/20
9216/8669 [===============================] - 22s - loss: 0.4593 - val_loss: 0.0658
Epoch 4/20
9216/8669 [===============================] - 22s - loss: 0.3268 - val_loss: 0.0644
Epoch 5/20
9216/8669 [===============================] - 22s - loss: 0.2437 - val_loss: 0.0613
Epoch 6/20
9006/8669 [===============================] - 25s - loss: 0.1902 - val_loss: 0.0667
Epoch 7/20
9216/8669 [===============================] - 23s - loss: 0.1446 - val_loss: 0.0591
Epoch 8/20
9216/8669 [===============================] - 23s - loss: 0.1227 - val_loss: 0.0612
Epoch 9/20
9216/8669 [===============================] - 24s - loss: 0.1175 - val_loss: 0.0663
Epoch 10/20
4608/8669 [==============>...............] - ETA: 8s - loss: 0.1017 

In [ ]:
import matplotlib.pyplot as plt

### print the keys contained in the history object
print(history_object.history.keys())
for key in history_object.history.keys():
    print ("key:", key,", values:", history_object.history[key])

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()